# Project: No-show Appointment Data Analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

The dataset collects information from 100k medical appointments in Brazil. A number of characteristics about the patient are included.
>
<li>'PatientID' identifies a specific patient.
<li>'AppointmentID' identifies a specific appointment.
<li>'Gender' indicates the gender of the patient.
<li>‘ScheduledDay’ tells us the date the patient set up their appointment.
<li>'AppointmentDay' tells us the date of the appointment.
<li>'Age' indicates the age of the patient.
<li>‘Neighborhood’ indicates the location of the hospital.
<li>‘Scholarship’ indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família.
<li>'Hipertension' indicates whether or not the patient is suffering from hipertension.
<li>'Diabetes' indicates whether or not the patient is suffering from diabetes.
<li>'Alcoholism' indicates whether or not the patient is suffering from alcoholism.
<li>'Handcap' indicates whether or not the patient has a disability.
<li>'SMS_received' tells us if the patient received an SMS upfront about the upcoming appointment.
<li>'No-show' indicates whether or not the patient showed up for their appointment.

This analysis focuses on data of patients that showed up or didn't show up for their medical appointments.
The intention is to find differences between the two groups.

### Questions
>
<li>Do no-shows for appointments differ depending on the age of the patient? Hypothesis: Older patients are more responsible and have a lower no-show rate.
<li>Do no-shows for appointments differ if patients received an SMS? Hypothesis: Patients that received an SMS are reminded of their appointment and have a lower no-show rate.
<li>Do no-shows for appointments differ if patients have a scholarship? Hypothesis: Patients with a scholarship have an insurance and therefore, don't need to worry about the expenses of an appointment and have a lower no-show rate.
<li>Do no-shows for appointments differ if the appointment is scheduled way in advance? Hypothesis: Patients that schedule their appointments way in advance, forget about them and have a higher no-show rate.

<a id='wrangling'></a>
## Data Wrangling

### General Properties

In [1]:
# import of packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [2]:
# load data 
df = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')

In [4]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   patient_id       110527 non-null  float64
 1   appointment_id   110527 non-null  int64  
 2   gender           110527 non-null  object 
 3   scheduled_day    110527 non-null  object 
 4   appointment_day  110527 non-null  object 
 5   age              110527 non-null  int64  
 6   neighbourhood    110527 non-null  object 
 7   scholarship      110527 non-null  int64  
 8   hipertension     110527 non-null  int64  
 9   diabetes         110527 non-null  int64  
 10  alcoholism       110527 non-null  int64  
 11  handicap         110527 non-null  int64  
 12  messaged         110527 non-null  int64  
 13  no_show          110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [40]:
df.describe()

,patient_id,appointment_id,age,scholarship,hipertension,diabetes,alcoholism,handicap,messaged
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [24]:
df.duplicated().sum()

0

In [33]:
column_list = df.columns
column_list = column_list.tolist()

In [41]:
df.duplicated(subset='patient_id').sum()

48228

In [46]:
for i in range(0, len(df.columns)):
    print(df.columns[i])
    print(df.duplicated(subset=df.columns[i]).sum())

patient_id
48228
appointment_id
0
gender
110525
scheduled_day
6978
appointment_day
110500
age
110423
neighbourhood
110446
scholarship
110525
hipertension
110525
diabetes
110525
alcoholism
110525
handicap
110522
messaged
110525
no_show
110525


In [47]:
#for i in range(0, len(df.columns)):
#    print(df.columns[i].value_counts())

### Data Cleaning 

<li> Adjust column names
<li> Adjust data types
<li> Find na values    

In [15]:
# adjust column names
df.columns
df.rename(columns={'PatientId':'patient_id', 'AppointmentID':'appointment_id', 'Gender':'gender', 
                   'ScheduledDay':'scheduled_day', 'AppointmentDay':'appointment_day', 'Age':'age', 
                   'Neighbourhood':'neighbourhood', 'Scholarship':'scholarship', 'Hipertension':'hipertension',
                   'Diabetes':'diabetes', 'Alcoholism':'alcoholism', 'Handcap':'handicap', 'SMS_received':'messaged',
                   'No-show':'no_show'}, inplace=True)
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


<a id='eda'></a>
## Exploratory Data Analysis

### Question 1

<a id='conclusions'></a>
## Conclusions

### Limitations



### Question 1:

Hypothesis: 

